# Data Exploration

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import duckdb
import pickle

In [2]:
# Display Options for Pandas
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = None

In [3]:
data_path = Path("../data")
data_path_string = data_path.resolve().as_posix()

### ETL and EDA

In [5]:
con = duckdb.connect(database=f"{data_path}/spotify.db")

In [6]:
def clean_db(con: duckdb.DuckDBPyConnection):
    con.execute("drop table if exists albums")
    con.execute("drop table if exists artists")
    con.execute("drop table if exists tracks")
    con.execute("drop table if exists features")
    con.execute("drop table if exists genres")

clean_db(con)

In [7]:
def load_data_into_db(con: duckdb.DuckDBPyConnection, data_path_string: str):
    con.read_csv(f"{data_path_string}/spotify_albums.csv", all_varchar=False).create("albums")
    con.read_csv(f"{data_path_string}/spotify_artists.csv", all_varchar=False).create("artists")
    con.read_csv(f"{data_path_string}/spotify_tracks.csv", all_varchar=False).create("tracks")

load_data_into_db(con, data_path_string)

In [8]:
con.table("albums").limit(1).df()

,column00,album_type,artist_id,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,total_tracks,track_id,track_name_prev,uri,type
0,0,single,3DiDSECUqqY1AuBP8qtaIa,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",{'spotify': 'https://open.spotify.com/album/1gAM7M4rBwEbSPeAQR2nx1'},https://api.spotify.com/v1/albums/1gAM7M4rBwEbSPeAQR2nx1,1gAM7M4rBwEbSPeAQR2nx1,"[{'height': 640, 'url': 'https://i.scdn.co/image/5872e4d8fac4ef7552576d481b1d676189b4056a', 'wid...",If I Ain't Got You EP,2019-02-08,day,6,2iejTMy9XZ8Gaae0aQ2yl0,track_32,spotify:album:1gAM7M4rBwEbSPeAQR2nx1,album


In [9]:
con.table("artists").limit(3).df()

,column0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type
0,0,44,23230,"['sertanejo', 'sertanejo pop', 'sertanejo tradicional', 'sertanejo universitario']",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist
1,1,22,313,[],1dLnVku4VQUOLswwDFvRc9,The Grenadines,4wqwj0gA8qPZKLl5WVqXml,track_30,artist
2,2,26,1596,['danish pop rock'],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist


In [10]:
con.table("tracks").limit(3).df()

,column00,acousticness,album_id,analysis_url,artists_id,available_markets,country,danceability,disc_number,duration_ms,energy,href,id,instrumentalness,key,liveness,loudness,lyrics,mode,name,playlist,popularity,preview_url,speechiness,tempo,time_signature,track_href,track_name_prev,track_number,uri,valence,type
0,0,0.294,0D3QufeCudpQANOR7luqdr,https://api.spotify.com/v1/audio-analysis/5qljLQuKnNJf4F4vfxQB0V,['3mxJuHRn2ZWD5OofvJtDZY'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",BE,0.698,1.0,235584.0,0.606,https://api.spotify.com/v1/tracks/5qljLQuKnNJf4F4vfxQB0V,5qljLQuKnNJf4F4vfxQB0V,0.000003,10.0,0.151,-7.447,"\r\n\r\nPerhaps I am bound to be restless\r\nAlways yearning, never satisfied\r\nPerhaps I'm ach...",0.0,Blood,Hipsteribrunssi,28.0,https://p.scdn.co/mp3-preview/1b05a902da3a251d07a38aa710ffae559fc33d08?cid=b3cdb16d0df2409abf6a8...,0.0262,115.018,4.0,https://api.spotify.com/v1/tracks/5qljLQuKnNJf4F4vfxQB0V,track_14,1.0,spotify:track:5qljLQuKnNJf4F4vfxQB0V,0.622,track
1,1,0.863,1bcqsH5UyTBzmh9YizdsBE,https://api.spotify.com/v1/audio-analysis/3VAX2MJdmdqARLSU5hPMpm,['4xWMewm6CYMstu0sPgd9jJ'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",BE,0.719,1.0,656960.0,0.308,https://api.spotify.com/v1/tracks/3VAX2MJdmdqARLSU5hPMpm,3VAX2MJdmdqARLSU5hPMpm,0.000000,6.0,0.253,-10.340,\r\nYour Gods and my Gods-do you or I know which are the stronger? Native Proverb.\r\nEAST of Su...,1.0,The Ugly Duckling,Animal Stories,31.0,https://p.scdn.co/mp3-preview/d8140736a6131cb5595f061975173a272c343a0a?cid=b3cdb16d0df2409abf6a8...,0.9220,115.075,3.0,https://api.spotify.com/v1/tracks/3VAX2MJdmdqARLSU5hPMpm,track_3,3.0,spotify:track:3VAX2MJdmdqARLSU5hPMpm,0.589,track
2,2,0.750,4tKijjmxGClg4JOLAyo2qE,https://api.spotify.com/v1/audio-analysis/1L3YAhsEMrGVvCgDXj2TYn,['3hYaK5FF3YAglCj5HZgBnP'],['GB'],BE,0.466,1.0,492840.0,0.931,https://api.spotify.com/v1/tracks/1L3YAhsEMrGVvCgDXj2TYn,1L3YAhsEMrGVvCgDXj2TYn,0.000000,4.0,0.938,-13.605,"\r\n\r\nClosed off from love, I didn't need the pain\r\nOnce or twice was enough and it was all ...",0.0,Jimmy Launches His Own Range Of Greetings Cards,Best Of British Comedy,31.0,https://p.scdn.co/mp3-preview/c8af28fb15185b18977152eb50eefef8d90af5a2?cid=b3cdb16d0df2409abf6a8...,0.9440,79.565,4.0,https://api.spotify.com/v1/tracks/1L3YAhsEMrGVvCgDXj2TYn,track_4,4.0,spotify:track:1L3YAhsEMrGVvCgDXj2TYn,0.085,track


In [44]:
# Construct Genre Table (with Polars because it's faster than pandas)
def create_genre_table(con: duckdb.DuckDBPyConnection):
    df = con.query("select track_id, genres from artists").pl()
    
    # Explode genre array into rows for each genre
    df = (
        df.with_columns(
            pl.col("genres").str.replace("^\[\]$", "NoGenre")
        ).with_columns(
            pl.col("genres").str.strip("[]").str.replace_all("'", "").str.split(", ")
        )
        .explode("genres")
        .unique()
    )

    # Explode each genre into words
    df = (
        df.with_columns(
            pl.col("genres").str.split(" ").alias("words")
        ).explode("words")
        .filter(pl.col("words") != "")
    )
    
    # Create Genre Table with Top 25 Words
    con.execute("drop table if exists genres")
    sql = """ 
        create table genres as
        with genres as (
            select * from df
        ),
        word_counts as (
            select 
                words, count(words)
            from genres
            group by words
            order by count(words) desc
            limit 25
        )
        select
            g.track_id, 
            g.genres, 
            case 
                when w.words is null then 'Other'
                when w.words = 'NoGenre' then 'Other'
                when w.words = 'hip' then 'hiphop'
                when w.words = 'hop' then 'hiphop'
                else w.words
            end as words
        from genres g
        left join word_counts w on g.words = w.words;
    """
    con.execute(sql)

create_genre_table(con)

In [37]:
con.table("genres").limit(3).df()

,track_id,genres,word
0,7sBDv4kLo2GVucy5VGh7GX,argentine indie,indie
1,0D2ZzEZ0DgOrWXuGyiUtSq,art pop,pop
2,0ZD0uZpctoES39tCWKrfQU,colombian pop,pop


In [45]:
# Construct Features Table
def create_features_table(con: duckdb.DuckDBPyConnection):
    sql = """
        drop table if exists features;
        create table features as
        select
            t.id, 
            t.acousticness, t.danceability, t.energy, t.instrumentalness, t.liveness, t.loudness, t.speechiness, t.tempo, t.valence,
            t.name as track_name, ar.name as artist_name, a.name as album_name, g.genres, g.words,
            t.preview_url, t.track_href, t.analysis_url
        from tracks t
        join albums a on t.album_id = a.id
        join artists ar on a.artist_id = ar.id
        join genres g on t.id = g.track_id
        """
    con.execute(sql)
    
create_features_table(con)

In [46]:
con.table("features").limit(3).df()

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_name,artist_name,album_name,genres,words,preview_url,track_href,analysis_url
0,3RNyGLgSvmVRZ7xKUp8Wgd,0.724,0.754,0.449,0.0,0.162,-14.013,0.0615,82.751,0.973,In the Summertime,Mungo Jerry,In the Summertime,british blues,blues,https://p.scdn.co/mp3-preview/ea8c3f26e654c14d732263d1503554863abc1ef6?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/3RNyGLgSvmVRZ7xKUp8Wgd,https://api.spotify.com/v1/audio-analysis/3RNyGLgSvmVRZ7xKUp8Wgd
1,7GmZAo7zyHrsZZR7n1YMGQ,0.133,0.665,0.720,0.0,0.297,-6.812,0.0534,145.015,0.736,Kuuletko,Dilemma,Kuuletko,finnish dance pop,dance,https://p.scdn.co/mp3-preview/f9d4bc5615dd48ac0d80b86bea4a2de11c75dbb7?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/7GmZAo7zyHrsZZR7n1YMGQ,https://api.spotify.com/v1/audio-analysis/7GmZAo7zyHrsZZR7n1YMGQ
2,7GmZAo7zyHrsZZR7n1YMGQ,0.133,0.665,0.720,0.0,0.297,-6.812,0.0534,145.015,0.736,Kuuletko,Dilemma,Kuuletko,finnish dance pop,pop,https://p.scdn.co/mp3-preview/f9d4bc5615dd48ac0d80b86bea4a2de11c75dbb7?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/7GmZAo7zyHrsZZR7n1YMGQ,https://api.spotify.com/v1/audio-analysis/7GmZAo7zyHrsZZR7n1YMGQ


In [47]:
con.query("summarize features").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,000YQJ9lmNX02OxJ7hEh4R,7zzpno7uAqkAzWZDQuGEFA,44144,NaN,NaN,NaN,NaN,NaN,177098,0.0%
1,acousticness,DOUBLE,0.0,0.996,4216,0.30929664225416953,0.33604490938544884,0.01925844643722915,0.1550135039236563,0.5682746634157062,177098,0.0%
2,danceability,DOUBLE,0.0,0.984,1092,0.5710494263063426,0.18656411979477777,0.4509322327745004,0.5919617059954096,0.712510255385757,177098,0.0%
3,energy,DOUBLE,0.0,1.0,2057,0.6178082640837254,0.25785957943271964,0.4575389675154186,0.6643883532123463,0.8257617931334229,177098,0.0%
4,instrumentalness,DOUBLE,0.0,1.0,5069,0.1791445404397574,0.31989396763871286,0.0,0.00025093863587464545,0.16486495461102174,177098,0.0%
5,liveness,DOUBLE,0.0,0.999,1659,0.18767907542716142,0.15966403788735137,0.09380909538021347,0.12212428659996234,0.23176282294120165,177098,0.0%
6,loudness,DOUBLE,-57.436,1.605,15865,-9.009515635410976,5.729097411102268,-10.577909039364432,-7.419136928263456,-5.390644740071115,177098,0.0%
7,speechiness,DOUBLE,0.0,0.964,1441,0.08850138002687677,0.10380906251766613,0.03660561838566085,0.04953575129580757,0.08946598755491234,177098,0.0%
8,tempo,DOUBLE,0.0,244.035,27649,119.63785338626207,29.0156144136555,97.17237497212695,119.7430380488535,136.7544281821855,177098,0.0%
9,valence,DOUBLE,0.0,0.993,1683,0.47839892777445253,0.2637072150287679,0.2606227492249429,0.4754086161248152,0.6919887952477576,177098,0.0%


In [48]:
# Count songs by genre word
sql = """ 
    select 
        words, count(words) as count
    from features
    group by words
    order by count(words) desc
"""
con.query(sql).df()

,words,count
0,Other,110814
1,pop,11359
2,rock,8609
3,hiphop,8004
4,indie,5686
5,house,3593
6,metal,2801
7,jazz,2221
8,alternative,2218
9,classical,2137


## ML

In [49]:
def load_data_into_pd(con: duckdb.DuckDBPyConnection) -> pd.DataFrame:
    return con.query("select * from features").df()

df = load_data_into_pd(con)

In [50]:
df.columns

Index(['id', 'acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'track_name',
       'artist_name', 'album_name', 'genres', 'words', 'preview_url',
       'track_href', 'analysis_url'],
      dtype='object')

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177098 entries, 0 to 177097
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                177098 non-null  object 
 1   acousticness      177098 non-null  float64
 2   danceability      177098 non-null  float64
 3   energy            177098 non-null  float64
 4   instrumentalness  177098 non-null  float64
 5   liveness          177098 non-null  float64
 6   loudness          177098 non-null  float64
 7   speechiness       177098 non-null  float64
 8   tempo             177098 non-null  float64
 9   valence           177098 non-null  float64
 10  track_name        177098 non-null  object 
 11  artist_name       177098 non-null  object 
 12  album_name        177098 non-null  object 
 13  genres            177098 non-null  object 
 14  words             177098 non-null  object 
 15  preview_url       177098 non-null  object 
 16  track_href        17

In [53]:
df.describe()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,177098.000000,177098.000000,177098.000000,177098.000000,177098.000000,177098.000000,177098.000000,177098.000000,177098.000000
mean,0.309297,0.571049,0.617808,0.179145,0.187679,-9.009516,0.088501,119.637853,0.478399
std,0.336045,0.186564,0.257860,0.319894,0.159664,5.729097,0.103809,29.015614,0.263707
min,0.000000,0.000000,0.000000,0.000000,0.000000,-57.436000,0.000000,0.000000,0.000000
25%,0.019300,0.452000,0.458000,0.000000,0.093800,-10.564000,0.036600,97.060000,0.260000
50%,0.155000,0.592000,0.664000,0.000243,0.122000,-7.419000,0.049500,119.946000,0.475000
75%,0.568000,0.712000,0.826000,0.164000,0.231000,-5.388000,0.089500,136.749000,0.692000
max,0.996000,0.984000,1.000000,1.000000,0.999000,1.605000,0.964000,244.035000,0.993000


In [29]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

def get_pipeline(df) -> Pipeline:
    num_cols = df.select_dtypes(       ############# <-------


    # Check if pipeline exists
    if Path(data_path / "pipeline.pkl").exists():
        with open(data_path / "pipeline.pkl", "rb") as f:
            return pickle.load(f)
    
    # Otherwise, create pipeline
    pipeline = Pipeline([ ("scaler", MinMaxScaler()) ])
    pipeline.fit(df.loc[:, df.columns != "id"])
    with open(data_path / "pipeline.pkl", "wb") as f:
        pickle.dump(pipeline, f)
        
    return pipeline

pipe = get_pipeline(df)

In [13]:
def preprocess_data(df: pd.DataFrame) -> np.ndarray:
    pipe = get_pipeline(df)
    X = pipe.transform(df.loc[:, df.columns != "id"])
    return X

X = preprocess_data(df)

In [27]:
from sklearn.neighbors import NearestNeighbors

def get_model_knn(X: np.ndarray or None = None) -> NearestNeighbors:
    # Check if model exists
    if Path(data_path / "knn.pkl").exists():
        with open(data_path / "knn.pkl", "rb") as f:
            return pickle.load(f)

    # Otherwise, create model
    knn = NearestNeighbors(n_neighbors=5, algorithm="ball_tree", metric="cosine")
    knn.fit(X)
    with open(data_path / "knn.pkl", "wb") as f:
        pickle.dump(knn, f)

    return knn

knn = get_model_knn(X)

In [12]:
# lookup ids
def lookup_ids(con: duckdb.DuckDBPyConnection, lookup_query: str) -> str:
    sql = """
      select *
      from lookup
      where 
        regexp_matches(lower(concat(track_name, ' ', artist_name)), $param)
      limit 10
    """
    return con.execute(sql, { "param": lookup_query.lower()}).fetch_df()

df_songs = lookup_ids(con, "like a prayer")
df_songs

,id,track_name,artist_name,album_name,preview_url,track_href,analysis_url
0,0OuGlX0EnsXQ4vvOunF9A3,Like a Prayer,Goshfather,Like a Prayer,https://p.scdn.co/mp3-preview/dbe9844b5b38df522bff03e411aedb6c52c87559?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/0OuGlX0EnsXQ4vvOunF9A3,https://api.spotify.com/v1/audio-analysis/0OuGlX0EnsXQ4vvOunF9A3
1,2v7ywbUzCgcVohHaKUcacV,Like a Prayer,Madonna,Like a Prayer,https://p.scdn.co/mp3-preview/274e7167b6222fc3f8e167bf036c7286bf265f2d?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/2v7ywbUzCgcVohHaKUcacV,https://api.spotify.com/v1/audio-analysis/2v7ywbUzCgcVohHaKUcacV
2,1z3ugFmUKoCzGsI6jdY4Ci,Like a Prayer,Madonna,Celebration (double disc version),https://p.scdn.co/mp3-preview/b56a70770267b00ccae13c2e8c8a34ed54627d02?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/1z3ugFmUKoCzGsI6jdY4Ci,https://api.spotify.com/v1/audio-analysis/1z3ugFmUKoCzGsI6jdY4Ci


In [13]:
# Select Song
df_test = df.query("id == '2v7ywbUzCgcVohHaKUcacV'")
df_test

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
85789,2v7ywbUzCgcVohHaKUcacV,0.432,0.66,0.629,0.000073,0.172,-12.359,0.0387,111.926,0.324


In [21]:
def make_recommendations(con: duckdb.DuckDBPyConnection, df: pd.DataFrame, id: str, n_neighbors: int = 10) -> pd.DataFrame:
    # Predict
    df_test = df.query(f"id == '{id}'")
    X_test = preprocess_data(df_test)
    knn = get_model_knn()
    distances, indices = knn.kneighbors(X_test, n_neighbors=n_neighbors)
    ids = df.loc[indices[0].tolist(), :]

    # Lookup song details
    sql = """
        select * 
        from lookup
        where id in (select id from ids)
    """
    details = con.query(sql).df()

    # Merge
    merge = ids.merge(details, how="left", on="id")
    return merge


In [22]:
make_recommendations(con, df, id="2v7ywbUzCgcVohHaKUcacV")


,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_name,artist_name,album_name,preview_url,track_href,analysis_url
0,2v7ywbUzCgcVohHaKUcacV,0.432,0.660,0.629,0.000073,0.1720,-12.359,0.0387,111.926,0.324,Like a Prayer,Madonna,Like a Prayer,https://p.scdn.co/mp3-preview/274e7167b6222fc3...,https://api.spotify.com/v1/tracks/2v7ywbUzCgcV...,https://api.spotify.com/v1/audio-analysis/2v7y...
1,7mrsLJSTzTMRQjEzWDAlOw,0.421,0.619,0.653,0.000080,0.1470,-8.791,0.0294,118.958,0.327,Painted In The Sound,Justus Proffit,L.A.'s Got Me Down,https://p.scdn.co/mp3-preview/d5277b8f48b35e00...,https://api.spotify.com/v1/tracks/7mrsLJSTzTMR...,https://api.spotify.com/v1/audio-analysis/7mrs...
2,0flFPv83zCKHwzuhlEDILy,0.416,0.633,0.682,0.000105,0.1480,-6.547,0.0359,114.999,0.324,Take Your Time,Vance Joy,Nation of Two,https://p.scdn.co/mp3-preview/6841b4be0f54da98...,https://api.spotify.com/v1/tracks/0flFPv83zCKH...,https://api.spotify.com/v1/audio-analysis/0flF...
3,5axqYa8jADY4TKO9jMr7Zt,0.503,0.607,0.664,0.002130,0.1150,-10.376,0.0314,110.010,0.330,Lloré,Monsieur Periné,Caja De Música (Edición Especial),https://p.scdn.co/mp3-preview/fc993da8974873c4...,https://api.spotify.com/v1/tracks/5axqYa8jADY4...,https://api.spotify.com/v1/audio-analysis/5axq...
4,0fqjy5gVNVmQQmxOLLfb3c,0.400,0.638,0.687,0.000114,0.1660,-6.556,0.0351,114.980,0.323,Take Your Time,Vance Joy,Nation Of Two,https://p.scdn.co/mp3-preview/c4defe3d34b142df...,https://api.spotify.com/v1/tracks/0fqjy5gVNVmQ...,https://api.spotify.com/v1/audio-analysis/0fqj...
5,68vdi4VhdQ3JTTRWPi5z6U,0.447,0.629,0.645,0.000074,0.0992,-10.735,0.1030,115.077,0.283,Electric (feat. Khalid) [Ryan Riback Remix],Alina Baraz,The Color Of You (Remixes),https://p.scdn.co/mp3-preview/506689b5b743e367...,https://api.spotify.com/v1/tracks/68vdi4VhdQ3J...,https://api.spotify.com/v1/audio-analysis/68vd...
6,4ZmAMOU0bcmrwwOvEK8aDT,0.407,0.626,0.574,0.000000,0.1700,-7.730,0.0300,119.531,0.272,Quién Diría,Ricardo Arjona,Canciones De Amor,https://p.scdn.co/mp3-preview/319ca1a607c2ce81...,https://api.spotify.com/v1/tracks/4ZmAMOU0bcmr...,https://api.spotify.com/v1/audio-analysis/4ZmA...
7,3mY5W2fJF9wKoy5d4IOtsZ,0.445,0.651,0.594,0.000000,0.1230,-5.879,0.0404,113.971,0.300,Anywhere With You (feat. Andie Nora),Wallaby,Anywhere With You (feat. Andie Nora),https://p.scdn.co/mp3-preview/8da4a78de402994d...,https://api.spotify.com/v1/tracks/3mY5W2fJF9wK...,https://api.spotify.com/v1/audio-analysis/3mY5...
8,0wl2rHiiQRHAW41UHYOI1u,0.414,0.660,0.599,0.012400,0.1270,-9.132,0.0271,135.066,0.336,Family Tapes,Dead Horses,Family Tapes,https://p.scdn.co/mp3-preview/9c1dbca4aac171c1...,https://api.spotify.com/v1/tracks/0wl2rHiiQRHA...,https://api.spotify.com/v1/audio-analysis/0wl2...
9,1lIz3Wn8b1lBsl0pvOpmU7,0.491,0.599,0.661,0.000002,0.1530,-8.140,0.0529,124.052,0.306,Watching You,Lea Rue,Watching You,https://p.scdn.co/mp3-preview/cda37d5344b7e0b3...,https://api.spotify.com/v1/tracks/1lIz3Wn8b1lB...,https://api.spotify.com/v1/audio-analysis/1lIz...


## Clean Up

In [6]:
con.close()